In [2]:
from langchain.embeddings.base import Embeddings
from tokenizers import Tokenizer
import onnxruntime as ort
import numpy as np


class OnnxEmbedder(Embeddings):
  def __init__(self, model_path: str, tokenizer_path: str):
      self.session = ort.InferenceSession(model_path)
      self.tokenizer = Tokenizer.from_file(tokenizer_path)

  def _embed(self, text: str):
      encoding = self.tokenizer.encode(text)
      input_ids = np.array([encoding.ids], dtype=np.int64)
      attention_mask = np.array([[1] * len(encoding.ids)], dtype=np.int64)

      outputs = self.session.run(
          None, {"input_ids": input_ids, "attention_mask": attention_mask}
      )

      raw_vector = outputs[0][0]
      norm_vector = raw_vector / (np.linalg.norm(raw_vector) + 1e-10)
      return norm_vector.tolist()

  def embed_documents(self, texts: list[str]) -> list[list[float]]:
      return [self._embed(text) for text in texts]

  def embed_query(self, text: str) -> list[float]:
      return self._embed(text)

In [2]:
import pandas as pd

df_raw = pd.read_csv("../../db/news_2023_2025_raw.csv")
df_raw.head(5)

df_metadata = pd.read_csv("../../db/news_2023_2025_metadata.csv")
df_metadata.head(5)

df_data = pd.merge(df_raw, df_metadata, on="news_id")
df_data.head(5)

,news_id,wdate,title,article,press,url,image,summary,stock_list,industry_list,impact_score
0,20250523_0002,2025-05-23 18:52:00,"[단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈",매각가 5000억 안팎 달할듯\n결제시장 내 입지강화 포석\n카카오페이 [사진 = ...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 국내 대표 전자결제사업자인 카...,"[""카카오페이""]","[""금융 지원 서비스업""]",0.33
1,20250523_0004,2025-05-23 18:33:00,"골드만삭스 차기 CEO, 이재용·김병주·이창용 만났다",[사진=연합뉴스]\n세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,세계 2위 투자은행(IB)인 골드만삭스의 사장 겸 최고운영책임자(COO)인 존 월드...,"[""삼성전자""]","[""통신 및 방송 장비 제조업""]",0.05
2,20250523_0007,2025-05-23 18:00:00,[단독] 전자결제 강자 카카오페이 쓱·스마일 페이 인수 추진,국내 대표 전자결제사업자인 카카오페이가 신세계이마트 산하 간편결제사업부 인수에 나섰...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...,23일 정보기술(IT)·투자은행(IB) 업계에 따르면 카카오페이가 SSG닷컴 쓱페이...,"[""카카오페이""]","[""금융 지원 서비스업""]",0.33
3,20250523_0010,2025-05-23 17:52:00,조현준 효성重 지분 4.9% 美 테크펀드 2600억 매각,조현준 효성 회장이 상속세 재원 마련을 위해 효성중공업 지분 4.90%를 미국의 글...,매일경제,https://n.news.naver.com/mnews/article/009/000...,https://ssl.pstatic.net/static.news/image/news...,23일 효성중공업은 조 회장이 상속세 재원을 마련하기 위해 시간 외 매매로 효성중 ...,"[""효성중공업""]","[""전동기, 발전기 및 전기 변환 · 공급 · 제어 장치 제조업""]",0.08
4,20250523_0011,2025-05-23 17:52:00,몸집 키우는 카카오…'간편결제 빅3' 흔드나,"카카오페이, 신세계 페이 품나\n이커머스 결제 강화 점유 확대\n압도적 1위 네이버...",매일경제,https://n.news.naver.com/mnews/article/009/000...,https://imgnews.pstatic.net/image/009/2025/05/...,카카오페이가 신세계 이마트 측에서 쓱페이·스마일페이 인수를 추진하고 나선 건 국내 ...,"[""카카오페이""]","[""금융 지원 서비스업""]",0.33


In [3]:
embedding = OnnxEmbedder(
    model_path="../../modelapi/models/kr_sbert_mean_onnx/kr_sbert.onnx",
    tokenizer_path="../../modelapi/models/kr_sbert_mean_onnx/tokenizer.json",
)

In [ ]:
from langchain.vectorstores import Chroma
from langchain.schema import Document

# 2. Document 리스트로 변환
documents = []
for _, row in df_data.iterrows():
    doc = Document(
        page_content=row["summary"],
        metadata={
            "news_id": row["news_id"],
            "wdate": row["wdate"],
        },
    )
    documents.append(doc)

# 4. Chroma 벡터DB 생성 (디스크 저장 가능)
vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding,
    persist_directory="./chroma_store",  # 저장 위치
)

# 5. 저장
vectordb.persist()

/var/folders/pl/c9208lms3f937xrsflvwrm640000gn/T/ipykernel_20046/3642437239.py:29: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [6]:
from langchain.vectorstores import Chroma

vectordb = Chroma(
    persist_directory="../../modelapi/db/chroma_store",
    embedding_function=embedding,
)

In [7]:
query_vector = embedding  # shape: (768,) or (1, 768)

# 검색
results = vectordb.similarity_search_with_score(
    "NH투자증권은 4일 서울시 영등포구 NH투자증권 본사에서 트래블월렛과 업무협약(MOU)을 체결하고 외환 및 글로벌 투자 서비스,",
    k=5,
)

for i, (doc, score) in enumerate(results):
    similarity = 1 - score
    print(f"\nTop {i+1}")
    print(f"유사도: {similarity:.4f}")
    print(f"요약: {doc.page_content}")
    print(f"기타: {doc.metadata}")


Top 1
유사도: 0.7633
요약: NH투자증권은 4일 서울시 영등포구 NH투자증권 본사에서 트래블월렛과 업무협약(MOU)을 체결하고 외환 및 글로벌 투자 서비스, 기업금융 자금조달에 대한 협력을 강화하기 위해 NH투자증권의 모바일 거래 플랫폼인 나무(Namuh)에서 환전 및 결제 서비스 이용이 가능하도록 구현했다.
기타: {'wdate': '2024-11-05 15:13:00', 'news_id': '20241105_0083'}

Top 2
유사도: 0.5317
요약: 교보증권이 지난 25일 서울 여의도 본사에서 교보DTS, 람다256과 토큰 증권 발행(STO)·유통 사업 추진을 위한 업무협약(MOU)을 체결하고 계좌관리기관 역할을 수행하며 블록체인 솔루션 기술개발 및 컨설팅을 담당한다.
기타: {'wdate': '2024-11-26 10:37:00', 'news_id': '20241126_0157'}

Top 3
유사도: 0.5217
요약: 카카오뱅크가 트래블월렛과 함께 간편한 환전과 편리한 결제 서비스를 앞세워 외화 서비스 '기본'앱으로 알려졌던 외화 충전·결제 서비스 핀테크사 '트래블 월렛'과 업무 협약을 체결했다고 11일 밝혔다.
기타: {'wdate': '2024-03-11 18:25:00', 'news_id': '20240311_0014'}

Top 4
유사도: 0.5132
요약: NH투자증권은 투자전문회사인 COMPANY H와 전략적 파트너십을 맺고 업무제휴 협약을 지난 4일 맺었는데, 이번 협약을 통해 인수금융, 프로젝트 파이낸싱 등 기업금융(IB) 딜에 공동 참여하는 방식으로 협력하기로 했다.
기타: {'news_id': '20240705_0107', 'wdate': '2024-07-05 10:24:00'}

Top 5
유사도: 0.4973
요약: 교보증권이 디지털금융 생태계 조성과 토큰증권(STO) 신사업을 영위하고자 S 디지털마켓츠(S DM)와 전략적 협업을 위한 양해각서(MOU)를 체결했다고 6일 밝혔으며 주요 협력 내용에는 디지털

In [27]:
res = [{"wdate": doc.metadata['wdate']} for doc in results]

AttributeError: 'tuple' object has no attribute 'metadata'

In [8]:
res

[]